<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Exploración DataVAULT
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>


## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Abril 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Consideraciones para la consulta en datos abiertos**|
|**Descripción y alcance**|Script para la consulta de bases y exploración de bases en el DATAVAULT.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC<td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

## Instalaciones previas
 
Para poder utilizar SQL en Python es necesario instalar lo siguiente, escoja el driver de acuerdo al sistema operativo de su equipo: 
 
 
**Driver Microsoft ODBC para SQL SERVER en windows**
 
Descargar e instalar el driver de la página [Microsoft ODBC](https://docs.microsoft.com/en-us/sql/connect/odbc/windows/system-requirements-installation-and-driver-files?view=sql-server-ver15#installing-microsoft-odbc-driver-for-sql-server) versión 
 17.
 
**Driver Microsoft ODBC para SQL SERVER en Linux o MacOs**
 
Descargar e instalar el driver de la página [Microsoft ODBC](https://docs.microsoft.com/en-us/sql/connect/odbc/linux-mac/system-requirements?view=sql-server-ver15) versión 17 deacuerdo a la versión de su sistema operativo.
Recuerde estar conectado a la VPN para poder accerder. 

**Paquete pyobdc**

Quite el comentario e inicie la instalación simple.

In [1]:
import pyodbc 
from dotenv import load_dotenv
import os
load_dotenv()

server = os.getenv("server")
username= os.getenv("username")
password= os.getenv("password")

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

ruta_DATA="../../../../DATA/"
ruta_MUESTRAS="../../../Sample_Data/Raw/MUESTRAS/ "

In [2]:
### Paquetes usados para la exploración de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')



## 3. DESCRIPCIÓN DEL INFORME / INSUMO

### 3.1. Objetivo

El objetivo de este cuaderno es mostrar las bases de datos disponibles en el DATAVAULT, como consultarlas y algunos usos simples de modelos de limpieza.

En particular de aqui saldran las siguientes tablas para poner a disposición del público a traves del portal de datos abiertos, www.datos.gov.co:

- **Tabla de procesos SECOP II**: Contiene la información de los procesos que se han llevado a cabo en el Sistema Electrónico de Contratación Pública SECOP II.

- **Tabla de contratos SECOP II**: Contiene la información de los contratos que se han llevado a cabo en el Sistema Electrónico de Contratación Pública SECOP II.


- **Tabla de procesos SECOP I**: Contiene la información de los procesos que se han llevado a cabo en el Sistema Electrónico de Contratación Pública SECOP I.

-  **Tabla de plan anual de adquisiciones**: Contiene la información de las adquisiciones prioyectadas por entidad.

## 4. ADQUISICIÓN DE LOS DATOS

### 4.1. Datos de procesos SECOP II

Para consultar los datos de procesos SECOP II, se debe ejecutar la siguiente consulta:

```sql
SELECT * FROM [CCE_Sandbox].[datavault].[SAT_Example_Table]
```


In [3]:
sat_company = pd.read_sql_query("SELECT * FROM [CCE_Sandbox].[datavault].[Sat_Company]", cnxn)


In [4]:

sat_company.to_parquet(ruta_DATA+"sat_company.parquet", engine='pyarrow')

In [5]:
sat_buyer_dossier = pd.read_sql_query("SELECT * FROM [CCE_Sandbox].[datavault].[Sat_BuyerDossier]", cnxn)


In [6]:
sat_buyer_dossier.to_parquet(ruta_DATA+"sat_buyer_dossier.parquet",engine='pyarrow')

In [7]:
sat_categorization = pd.read_sql_query("SELECT * FROM [CCE_Sandbox].[datavault].[Sat_Categorization]", cnxn)


In [8]:
sat_categorization.to_parquet(ruta_DATA+"sat_categorization.parquet",engine='pyarrow')


In [3]:
## sat_procedure_request lo bajaremos en bloques de 1000000

n=pd.read_sql_query("SELECT COUNT(*) FROM [CCE_Sandbox].[datavault].[Sat_ProcedureRequest]", cnxn).values[0][0]
list_sat_procedure_request=[]
for i in range(0,n,500000):
    print("Inicia la consulta en", i)
    sat_procedure_request_temp = pd.read_sql_query("SELECT * FROM [CCE_Sandbox].[datavault].[Sat_ProcedureRequest] ORDER BY [HubProcedureRequestHashKey] OFFSET "+str(i)+" ROWS FETCH NEXT 1000000 ROWS ONLY", cnxn)
    list_sat_procedure_request.append(sat_procedure_request_temp)

Inicia la consulta en 0
Inicia la consulta en 500000
Inicia la consulta en 1000000
Inicia la consulta en 1500000
Inicia la consulta en 2000000
Inicia la consulta en 2500000
Inicia la consulta en 3000000


In [4]:
sat_procedure_request=pd.concat(list_sat_procedure_request)
sat_procedure_request.to_parquet(ruta_DATA+"sat_procedure_request.parquet",engine='pyarrow')    

In [5]:
### cerramos la conexión
cnxn.close()

## Carga de datos usando Python desde la ruta DATA

Cargamos la base desde la ruta DATA




In [1]:
import pandas as pd
ruta_DATA="../../../../DATA/"


In [2]:
sat_company = pd.read_parquet(ruta_DATA+"sat_company.parquet",engine="pyarrow")
sat_buyer_dossier = pd.read_parquet(ruta_DATA+"sat_buyer_dossier.parquet",engine="pyarrow")
sat_categorization = pd.read_parquet(ruta_DATA+"sat_categorization.parquet",engine="pyarrow")
sat_procedure_request = pd.read_parquet(ruta_DATA+"sat_procedure_request.parquet",engine="pyarrow")

ArrowInvalid: Casting from timestamp[us] to timestamp[ns] would result in out of bounds timestamp: 21116350800000000

In [ ]:

### Para el repositorio generamos muestras de los dataframes y los guardamos en archivos csv

sat_company.sample(100).to_csv(ruta_DATA+"sat_company.csv")
sat_buyer_dossier.sample(100).to_csv(ruta_DATA+"sat_buyer_dossier.csv")
sat_categorization.sample(100).to_csv(ruta_DATA+"sat_categorization.csv")
sat_procedure_request.sample(100).to_csv(ruta_DATA+"sat_procedure_request.csv")

In [ ]:

# unir las tablas utilizando las claves
procesos = pd.merge(sat_buyer_dossier, sat_procedure_request, on='BuyerDossierUniqueIdentifier', how='left')
proc_comp_buyer = pd.merge(procesos, sat_categorization, on='HubProcedureRequestHashKey', how='left')
proc_comp_buyer_cat = pd.merge(proc_comp_buyer, sat_categorization, on='HubProcedureRequestHashKey', how='left')


result = proc_comp_buyer_cat[columns]
